In [ ]:
import sys
import os
import json
import glob
import numpy as np
from lxml import etree
from pyxodr.road_objects.road import Road
from pyxodr.road_objects.lane import Lane, ConnectionPosition, LaneOrientation, TrafficOrientation
from pyxodr.road_objects.junction import Junction
from pyxodr.road_objects.lane_section import LaneSection
from pyxodr.road_objects.network import RoadNetwork
from shapely.geometry import Polygon
from enum import IntEnum

In [ ]:
carla_map_dir = "C:\\Carla-0.10.0\\CarlaUnreal\\Content\\Carla\\Maps\\OpenDrive"
odr_file = os.path.join(carla_map_dir, "Town04.xodr")

road_network = RoadNetwork(xodr_file_path=odr_file)

In [ ]:
roads = road_network.get_roads()
print(f"Number of roads in the network: {len(roads)}")
print(f"Type: {type(roads[0])}\nRoads: {roads}")

In [ ]:
multi_lane_section_roads = []

for road in roads:
    if len(road.lane_sections) > 1:
        multi_lane_section_roads.append(road)

print(f"Number of roads with multiple lane sections: {len(multi_lane_section_roads)}")
print("Road IDs with multiple lane sections:")
for road in multi_lane_section_roads:
    print(f"  - {road.id}")

# Check for no links case in lanes of roads with multiple lane sections
for road in multi_lane_section_roads:
    for lane_section in road.lane_sections:
        for lane in lane_section.left_lanes:
            if lane.lane_xml.attrib["type"] == "driving":
                link_xml = lane.lane_xml.find("link")
                if link_xml is None:
                    print(
                        f"  - Road ID: {road.id}, Lane Section ID: {lane_section.lane_section_ordinal}, Lane ID: {lane.id} has no link."
                    )
        for lane in lane_section.right_lanes:
            if lane.lane_xml.attrib["type"] == "driving":
                link_xml = lane.lane_xml.find("link")
                if link_xml is None:
                    print(
                        f"  - Road ID: {road.id}, Lane Section ID: {lane_section.lane_section_ordinal}, Lane ID: {lane.id} has no link."
                    )

Conclusion: There is no driving lane without a link in a road that has multiple lane_sections

In [ ]:
# Finding multi-successor/predecessor roads

multi_successor_roads = []
multi_predecessor_roads = []

for road in road_network.get_roads():
    if len(road.road_xml.find("link").findall("successor")) > 1:
        multi_successor_roads.append(road)
    if len(road.road_xml.find("link").findall("predecessor")) > 1:
        multi_predecessor_roads.append(road)

print(f"Multi-successor roads: {[road.id for road in multi_successor_roads]}")
print(f"Multi-predecessor roads: {[road.id for road in multi_predecessor_roads]}")

# Finding multi-successor/predecessor lanes

multi_successor_lanes = []
multi_predecessor_lanes = []

for road in road_network.get_roads():
    for lane_section in road.lane_sections:
        for lane in lane_section.left_lanes + lane_section.right_lanes:
            if lane.lane_xml.find("link") is not None:
                if lane.lane_xml.find("link").findall("successor") is not []:
                    if len(lane.lane_xml.find("link").findall("successor")) > 1:
                        multi_successor_lanes.append((lane.id, lane_section.lane_section_ordinal, road.id))
                if lane.lane_xml.find("link").findall("predecessor") is not []:
                    if len(lane.lane_xml.find("link").findall("predecessor")) > 1:
                        multi_predecessor_lanes.append((lane.id, lane_section.lane_section_ordinal, road.id))

print(f"Multi-successor lanes: {multi_successor_lanes}")
print(f"Multi-predecessor lanes: {multi_predecessor_lanes}")

Conclusion: There is no road element or lane that has multiple successor or predecessor links

In [ ]:
def get_road(road_network, road_id):
    for road in road_network.get_roads():
        if road.id == road_id:
            return road
    return None

In [ ]:
# Check if every pred of a road has itself as a succ/pred in the pred_road
for road_obj in road_network.get_roads():
    if road_obj.road_xml.attrib["junction"] != "-1":
        continue
    if road_obj.road_xml.find("link") is not None and road_obj.road_xml.find("link").findall("predecessor") is not []:
        for pred in road_obj.road_xml.find("link").findall("predecessor"):
            found = False
            if pred.attrib["elementType"] != "road":
                continue
            pred_road = get_road(road_network, pred.attrib["elementId"])
            if pred_road is not None:
                if (
                    pred_road.road_xml.find("link") is not None
                    and pred_road.road_xml.find("link").findall("successor") is not []
                ):
                    for succ in pred_road.road_xml.find("link").findall("successor"):
                        if succ.attrib["elementType"] == "road" and succ.attrib["elementId"] == road_obj.id:
                            found = True
                            break
                if (
                    pred_road.road_xml.find("link") is not None
                    and pred_road.road_xml.find("link").findall("predecessor") is not []
                ):
                    for pred_pred in pred_road.road_xml.find("link").findall("predecessor"):
                        if pred_pred.attrib["elementType"] == "road" and pred_pred.attrib["elementId"] == road_obj.id:
                            found = True
                            break
                if not found:
                    print(
                        f"Road {pred_road.id} is a predecessor of {road_obj.id}, but {pred_road.id} does not list {road_obj.id} as a successor/predecessor."
                    )
            else:
                print(f"Predecessor road with ID {pred.attrib['elementId']} not found in the network.")

Conclusion: Every predecessor of a road has itself as a successor/predecessor in the pred_road, and vice-versa for successor

Roads Data Analysis in JSON

In [ ]:
json_dir = "../carla"  # replace with your directory path
json_files = glob.glob(os.path.join(json_dir, "*.json"))

for jf in json_files:
    with open(jf, "r") as f:
        print(f"\nReading JSON file: {jf}")
        data = json.load(f)
        if "roads" in data:
            num_geometries = [len(road["geometry"]) for road in data["roads"]]
            print(f"Number of roads in the JSON: {len(data['roads'])}")
            print(f"Max geometries in a road: {max(num_geometries)}")
            print(f"Min geometries in a road: {min(num_geometries)}")
            print(f"Total geometries in the JSON: {sum(num_geometries)}")
            print(f"Average geometries per road: {sum(num_geometries) / len(num_geometries)}")
        else:
            print("No 'roads' key found in this JSON file.")